In [1]:
import sys
sys.path.append("..") 

In [2]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from pathlib import Path
import os
import torch.cuda
import albumentations as albu
import segmentation_models_pytorch as smp
from  segmentation_models_pytorch.utils.base import Metric
from segmentation_models_pytorch.base.modules import Activation
from collections import defaultdict

In [3]:
from Utils.dataset_utils import *
from Utils.visualization_utils import *

In [4]:
from torchsummary import summary
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset

In [5]:
DATASET_2D_BASE_PATH=Path(r'C:\Users\lm3088\Documents\POM-CTproject\Muhammad\MiceCT_2Daxials\Axials')
WIDTH=320
HEIGHT=320

In [6]:


def get_training_augmentation():
    train_transform = [

        albu.Resize(HEIGHT, WIDTH),
        albu.ShiftScaleRotate(scale_limit=0.20, rotate_limit=30, shift_limit=0.1, p=1, border_mode=cv2.BORDER_CONSTANT),
        albu.RandomCrop(height=320, width=320),
        albu.Blur(blur_limit=3, p=0.4),
        albu.GaussNoise(p=0.5),
        albu.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
        albu.RandomBrightness(p=0.75)

    ]
    return albu.Compose(train_transform)


def get_validation_augmentation():
    test_transform = [
        albu.Resize(512, 512)
    ]
    return albu.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform

    Args:
        preprocessing_fn (callbale): data normalization function
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose

    """

    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

In [7]:
df_train=load_data(DATASET_2D_BASE_PATH/'train')
df_val=load_data(DATASET_2D_BASE_PATH/'val')
df_train.head()

,images,masks
0,C:\Users\lm3088\Documents\POM-CTproject\Muhamm...,C:\Users\lm3088\Documents\POM-CTproject\Muhamm...
1,C:\Users\lm3088\Documents\POM-CTproject\Muhamm...,C:\Users\lm3088\Documents\POM-CTproject\Muhamm...
2,C:\Users\lm3088\Documents\POM-CTproject\Muhamm...,C:\Users\lm3088\Documents\POM-CTproject\Muhamm...
3,C:\Users\lm3088\Documents\POM-CTproject\Muhamm...,C:\Users\lm3088\Documents\POM-CTproject\Muhamm...
4,C:\Users\lm3088\Documents\POM-CTproject\Muhamm...,C:\Users\lm3088\Documents\POM-CTproject\Muhamm...


In [8]:
class DCS(Metric):
    __name__ = 'DCS'

    def __init__(self, eps=0.00001, activation=None, ignore_channels=None, **kwargs):
        super().__init__(**kwargs)
        self.eps = eps
     
        self.activation = Activation(activation)
        self.ignore_channels = ignore_channels

    def forward(self, y_pr, y_gt):
        y_pr = self.activation(y_pr)
        dice_numerator = 2 * torch.sum(y_pr * y_gt) + self.eps
        dice_denominator = torch.sum(y_pr) + torch.sum(y_gt) + self.eps
        dice_coefficient = dice_numerator / dice_denominator
        return dice_coefficient


### Training

In [9]:
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multiclass segmentation
DEVICE = 'cuda'
TRAIN_RUNS_PATH=r'C:\Users\lm3088\Documents\GitHub\MicroCTsegmentation\runs'
MODEL_NAME='Unet'
BATCH_SIZE=8
EPOCHS=100


In [10]:
ENCODERS = ['se_resnext50_32x4d','efficientnet-b2','resnet101','resnet34','densenet121']

In [11]:
for j in range(len(ENCODERS)):
    
    model = smp.Unet(
    encoder_name=ENCODERS[j], 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=1, 
    activation=ACTIVATION,
    )
    preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODERS[j], ENCODER_WEIGHTS)
    
    WEIGHTS_PATH=os.path.join(TRAIN_RUNS_PATH,f'{MODEL_NAME}_{ENCODERS[j]}')
    if os.path.exists(WEIGHTS_PATH)==False:
        os.mkdir(WEIGHTS_PATH)
    else:
        print(f"Warning! Directory {WEIGHTS_PATH } already exists")
    
    train_dataset = Dataset(
    df_train['images'], 
    df_train['masks'], 
    augmentation=get_training_augmentation(), 
    preprocessing=get_preprocessing(preprocessing_fn)
    )

    valid_dataset = Dataset(
        df_val['images'], 
        df_val['masks'], 
        augmentation=get_validation_augmentation(), 
        preprocessing=get_preprocessing(preprocessing_fn)
    ) 

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    loss = smp.utils.losses.DiceLoss()
    metrics = [
        smp.utils.metrics.IoU(threshold=0.5),
        DCS()

    ]

    optimizer = torch.optim.Adam([ 
        dict(params=model.parameters(), lr=0.0001),
    ])
    train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
    )

    valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
    )
    min_loss = 100000000
    train_history=defaultdict(list)
    valid_history=defaultdict(list)

    for i in range(0, EPOCHS):

        print('\nEpoch: {}'.format(i))
        train_logs = train_epoch.run(train_loader)
        valid_logs = valid_epoch.run(valid_loader)

        # do something (save model, change lr, etc.)
        if min_loss > valid_logs['dice_loss']:
            min_loss = valid_logs['dice_loss']
            torch.save(model, os.path.join(WEIGHTS_PATH,f'best_{str(i)}_{round(min_loss,4)}.pt'))
            print('Model saved!')

        if i == 25:
            optimizer.param_groups[0]['lr'] = 1e-5
            print('Decrease decoder learning rate to 1e-5!')
        # Maintain History
        for log_key in train_logs.keys():
            train_history[log_key].append(train_logs[log_key])
            valid_history[log_key].append(valid_logs[log_key])
    pd.DataFrame(valid_history).to_csv(os.path.join(WEIGHTS_PATH,'validation_logs.csv'))
    pd.DataFrame(train_history).to_csv(os.path.join(WEIGHTS_PATH,'train_logs.csv'))
    print(f"{WEIGHTS_PATH} Completed!!!")

Warning! Directory C:\Users\lm3088\Documents\GitHub\MicroCTsegmentation\runs\Unet_se_resnext50_32x4d already exists


C:\Users\lm3088\Anaconda3\envs\condaenv\lib\site-packages\albumentations\augmentations\transforms.py:1800: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(



Epoch: 0
valid: 100%|███████████████████| 192/192 [00:43<00:00,  4.36it/s, dice_loss - 0.7285, iou_score - 0.5773, DCS - 0.2715]
Model saved!

Epoch: 1
valid: 100%|███████████████████| 192/192 [00:43<00:00,  4.38it/s, dice_loss - 0.6946, iou_score - 0.6369, DCS - 0.3051]
Model saved!

Epoch: 2
valid: 100%|████████████████████| 192/192 [00:44<00:00,  4.35it/s, dice_loss - 0.6799, iou_score - 0.698, DCS - 0.3195]
Model saved!

Epoch: 3
valid: 100%|███████████████████| 192/192 [00:44<00:00,  4.34it/s, dice_loss - 0.6916, iou_score - 0.6997, DCS - 0.3071]

Epoch: 4
valid: 100%|████████████████████| 192/192 [00:43<00:00,  4.37it/s, dice_loss - 0.659, iou_score - 0.7301, DCS - 0.3389]
Model saved!

Epoch: 5
valid: 100%|███████████████████| 192/192 [00:44<00:00,  4.33it/s, dice_loss - 0.6894, iou_score - 0.6635, DCS - 0.3073]

Epoch: 6
valid: 100%|███████████████████| 192/192 [00:44<00:00,  4.34it/s, dice_loss - 0.7098, iou_score - 0.6617, DCS - 0.2846]

Epoch: 7
valid: 100%|████████████████

valid: 100%|████████████████████| 192/192 [00:43<00:00,  4.46it/s, dice_loss - 0.1943, iou_score - 0.7648, DCS - 0.372]

Epoch: 64
valid: 100%|█████████████████████| 192/192 [00:43<00:00,  4.46it/s, dice_loss - 0.184, iou_score - 0.774, DCS - 0.3705]
Model saved!

Epoch: 65
valid: 100%|███████████████████| 192/192 [00:43<00:00,  4.43it/s, dice_loss - 0.1929, iou_score - 0.7673, DCS - 0.3697]

Epoch: 66
valid: 100%|███████████████████| 192/192 [00:43<00:00,  4.45it/s, dice_loss - 0.1932, iou_score - 0.7652, DCS - 0.3672]

Epoch: 67
valid: 100%|███████████████████| 192/192 [00:43<00:00,  4.46it/s, dice_loss - 0.1945, iou_score - 0.7577, DCS - 0.3711]

Epoch: 68
valid: 100%|███████████████████| 192/192 [00:43<00:00,  4.46it/s, dice_loss - 0.2101, iou_score - 0.7526, DCS - 0.3706]

Epoch: 69
valid: 100%|████████████████████| 192/192 [00:43<00:00,  4.46it/s, dice_loss - 0.2032, iou_score - 0.7548, DCS - 0.368]

Epoch: 70
valid: 100%|███████████████████| 192/192 [00:43<00:00,  4.45it/s, dice

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b2-8bb594d6.pth" to C:\Users\lm3088/.cache\torch\hub\checkpoints\efficientnet-b2-8bb594d6.pth


...

valid: 100%|███████████████████| 192/192 [00:36<00:00,  5.20it/s, dice_loss - 0.1826, iou_score - 0.7598, DCS - 0.3744]

Epoch: 64
valid: 100%|███████████████████| 192/192 [00:36<00:00,  5.22it/s, dice_loss - 0.1805, iou_score - 0.7592, DCS - 0.3742]

Epoch: 65
valid: 100%|███████████████████| 192/192 [00:36<00:00,  5.22it/s, dice_loss - 0.1893, iou_score - 0.7488, DCS - 0.3741]

Epoch: 66
valid: 100%|███████████████████| 192/192 [00:36<00:00,  5.20it/s, dice_loss - 0.1673, iou_score - 0.7747, DCS - 0.3738]
Model saved!

Epoch: 67
valid: 100%|█████████████████████| 192/192 [00:36<00:00,  5.20it/s, dice_loss - 0.194, iou_score - 0.7448, DCS - 0.375]

Epoch: 68
valid: 100%|███████████████████| 192/192 [00:37<00:00,  5.19it/s, dice_loss - 0.2849, iou_score - 0.6566, DCS - 0.3756]

Epoch: 69
valid: 100%|███████████████████| 192/192 [00:37<00:00,  5.18it/s, dice_loss - 0.2285, iou_score - 0.7099, DCS - 0.3764]

Epoch: 70
valid: 100%|███████████████████| 192/192 [00:36<00:00,  5.21it/s, dice

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/densenet121-fbdb23505.pth" to C:\Users\lm3088/.cache\torch\hub\checkpoints\densenet121-fbdb23505.pth


  0%|          | 0.00/30.9M [00:00<?, ?B/s]


Epoch: 0
valid: 100%|███████████████████| 192/192 [00:42<00:00,  4.52it/s, dice_loss - 0.7796, iou_score - 0.6158, DCS - 0.2203]
Model saved!

Epoch: 1
valid: 100%|███████████████████| 192/192 [00:42<00:00,  4.52it/s, dice_loss - 0.6797, iou_score - 0.5977, DCS - 0.3202]
Model saved!

Epoch: 2
valid: 100%|████████████████████| 192/192 [00:42<00:00,  4.52it/s, dice_loss - 0.664, iou_score - 0.6239, DCS - 0.3357]
Model saved!

Epoch: 3
valid: 100%|███████████████████| 192/192 [00:42<00:00,  4.53it/s, dice_loss - 0.6917, iou_score - 0.6952, DCS - 0.3075]

Epoch: 4
valid: 100%|████████████████████| 192/192 [00:42<00:00,  4.53it/s, dice_loss - 0.644, iou_score - 0.7396, DCS - 0.3545]
Model saved!

Epoch: 5
valid: 100%|████████████████████| 192/192 [00:42<00:00,  4.52it/s, dice_loss - 0.6676, iou_score - 0.4907, DCS - 0.331]

Epoch: 6
valid: 100%|███████████████████| 192/192 [00:42<00:00,  4.52it/s, dice_loss - 0.7362, iou_score - 0.5014, DCS - 0.2609]

Epoch: 7
valid: 100%|████████████████

valid: 100%|████████████████████| 192/192 [00:42<00:00,  4.53it/s, dice_loss - 0.221, iou_score - 0.7352, DCS - 0.3695]

Epoch: 64
valid: 100%|███████████████████| 192/192 [00:42<00:00,  4.50it/s, dice_loss - 0.2152, iou_score - 0.7441, DCS - 0.3733]

Epoch: 65
valid: 100%|████████████████████| 192/192 [00:42<00:00,  4.49it/s, dice_loss - 0.2088, iou_score - 0.7458, DCS - 0.371]

Epoch: 66
valid: 100%|███████████████████| 192/192 [00:42<00:00,  4.48it/s, dice_loss - 0.2007, iou_score - 0.7572, DCS - 0.3718]

Epoch: 67
valid: 100%|███████████████████| 192/192 [00:42<00:00,  4.48it/s, dice_loss - 0.1795, iou_score - 0.7754, DCS - 0.3688]
Model saved!

Epoch: 68
valid: 100%|███████████████████| 192/192 [00:42<00:00,  4.55it/s, dice_loss - 0.1905, iou_score - 0.7611, DCS - 0.3707]

Epoch: 69
valid: 100%|████████████████████| 192/192 [00:43<00:00,  4.44it/s, dice_loss - 0.1848, iou_score - 0.766, DCS - 0.3697]

Epoch: 70
valid: 100%|███████████████████| 192/192 [00:41<00:00,  4.60it/s, dice